In [ ]:
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import IPython.display as ipd
from sklearn import preprocessing 
import winsound
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
model_type = []
maneuvering_direction = []
fault = []
ID = []

for file in os.listdir('./Dataset-ArIES/drone_A/A/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset-ArIES/drone_B/B/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset-ArIES/drone_C/C/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)

In [ ]:
for file in os.listdir('./Dataset/drone_A/A/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset/drone_B/B/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset/drone_C/C/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)

In [ ]:
data = np.array([ID, model_type, maneuvering_direction, fault])
df = pd.DataFrame(data.T, columns=['ID', 'model_type', 'maneuvering_direction', 'fault']) 
df = pd.get_dummies(df, columns=['model_type', 'maneuvering_direction', 'fault'], dtype=int)
df.to_csv('train.csv', index = False)
df

In [ ]:
mic1 = []
mic2 = []
for file in os.listdir('./Dataset/drone_A/A/train/mic1'):
    filename = os.path.join('./Dataset/drone_A/A/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_A/A/train/mic2'):
    filename = os.path.join('./Dataset/drone_A/A/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)

for file in os.listdir('./Dataset/drone_B/B/train/mic1'):
    filename = os.path.join('./Dataset/drone_B/B/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_B/B/train/mic2'):
    filename = os.path.join('./Dataset/drone_B/B/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)

for file in os.listdir('./Dataset/drone_C/C/train/mic1'):
    filename = os.path.join('./Dataset/drone_C/C/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_C/C/train/mic2'):
    filename = os.path.join('./Dataset/drone_C/C/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)
print("Data loaded")

In [ ]:
def compute_sg(audio_data, sr=22050):
    spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    # mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, S=spectrogram, n_mfcc=30)
    # for i in range(len(spectrogram)):
    #     for j in range(len(spectrogram[i])):
    #           spectrogram[i][j] = spectrogram[i][j]/np.max(abs(spectrogram[i]))
    return spectrogram

In [ ]:
train_array = []
for i in tqdm(range(len(mic1))):
    train_array.append(compute_sg(mic1[i]))
for i in tqdm(range(len(mic1))):
    train_array.append(compute_sg(mic2[i]))
train_array = np.array(train_array)

In [ ]:
train_array_2d = train_array.reshape(len(train_array), train_array.shape[1]*train_array.shape[2])
np.savetxt('sg_train.csv', train_array_2d, delimiter=',')

In [ ]:
mic1_test = []
mic2_test = []
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_A/A/test/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1_test.append(audio_data)
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_A/A/test/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2_test.append(audio_data)

for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_B/B/test/mic1', file)
    y, _ = librosa.load(filename)
    mic1_test.append(y)
for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_B/B/test/mic2', file)
    y, _ = librosa.load(filename)
    mic2_test.append(y)

for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_C/C/test/mic1', file)
    y, _ = librosa.load(filename)
    mic1_test.append(y)
for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_C/C/test/mic2', file)
    y, _ = librosa.load(filename)
    mic2_test.append(y)

print('Data loaded')

In [ ]:
sg_array_test = []
for i in tqdm(range(len(mic1_test))):
    sg_array_test.append(compute_sg(mic1_test[i]))
for i in tqdm(range(len(mic2_test))):
    sg_array_test.append(compute_sg(mic2_test[i]))
sg_array_test = np.array(sg_array_test)

In [ ]:
sg_array_test_2d = sg_array_test.reshape(sg_array_test.shape[0], sg_array_test.shape[1]*sg_array_test.shape[2])
np.savetxt('sg_test.csv', sg_array_test_2d, delimiter=',')